# Praktikum IF3270 2023/2024

Tujuan praktikum IF3270 Pembelajaran Mesin:
1.   Peserta memahami rangkaian proses analitik data menggunakan pendekatan pembelajaran mesin. 
2.   Peserta memahami bahwa proses pengembangan model pembelajaran mesin juga ditentukan dari kualitas data, penanganan data, dan penentuan algoritma serta hyperparameter-nya; tidak cukup hanya dengan memastikan implementasi algoritma berjalan tanpa kesalahan.
3.   Peserta mampu menginterpretasikan hasil dari evaluasi model dalam proses analitik menggunakan pendekatan pembelajaran mesin.

Praktikum dilaksanakan secara berkelompok. Setiap kelompok terdiri atas 2 mahasiswa. Perhatikan bahwa terdapat berkas yang harus dikumpulkan saat keberjalanan praktikum untuk bagian A (25 April 2024, pukul 12.00 WIB) dan berkas yang dikumpulkan setelah waktu praktikum selesai untuk bagian B (25 April 2024, pukul 21.00 WIB).

Disediakan data yang sudah dibagi menjadi data latih (`df_train`), data validasi (`df_val`), dan data uji (`df_test`).

**Bagian 1**: (batas waktu: 25 April 2024, 12.00 WIB)

1. Buatlah _baseline_ dengan menggunakan model _logistic regression_.
2. Lakukan analisis data terkait hal berikut:
    - _duplicate value_,
    - _missing value_,
    - _outlier_,
    - _balance of data_.
3. Jelaskan rencana penanganan yang ada pada poin 2.
4. Jelaskan teknik _encoding_ yang digunakan terhadap data yang disediakan apabila dilakukan, disertai dengan alasan.
5. Buatlah desain eksperimen dengan menentukan hal berikut:
    - tujuan eksperimen,
    - variabel dependen dan independen,
    - strategi eksperimen,
    - skema validasi.
    
**Bagian 2**: (batas waktu: 25 April 2024, 21.00 WIB)

6. Implementasikan strategi eksperimen dan skema validasi yang telah ditentukan pada poin 5.
7. Berdasarkan hasil prediksi yang dihasilkan, buatlah kesimpulan analisis **hasil diabetes**.

---
Catatan:
- Jika terdapat perubahan jawaban pada poin 1—5 (contoh: perbedaan penanganan _outlier_), jelaskan pada laporan mengenai jawaban sebelum, jawaban sesudah, dan alasan pengubahan jawaban.
- Eksperimen dapat berupa penggantian model klasifikasi, pengaturan hyperparameter, model stacking, grid search, oversampling, undersampling, dan lain sebagainya. Semakin variatif eksperimen yang dilakukan, semakin baik.

## Dataset
`diabetes.csv` merupakan dataset yang telah dimodifikasi dari [Diabetes Health Indicators Dataset](https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset/) sebagai kumpulan indikator individu yang diperoleh dari survei untuk kasus diabetes. Dataset ini berguna untuk melakukan prediksi diabetes sehingga suatu individu dapat diketahui memiliki risiko tinggi diabetes atau tidak. Hal ini diperoleh dari fitur-fitur yang dapat dianalisis lebih lanjut sebelum mencapai kesimpulan.

Berikut adalah deskripsi singkat setiap kolom:

1. **HighBP**: Memiliki tekanan darah tinggi (BP: Blood Pressure) atau tidak
2. **HighChol**: Kolesterol tinggi atau tidak
3. **BMI**: Besaran Body Mass Index
4. **Smoker**: Perokok atau bukan perokok
5. **Stroke**: Pernah mengalami struk atau tidak
6. **HeartDiseaseorAttack**: Memiliki riwayat penyakit antara jantung koroner dan serangan jantung atau tidak sama sekali
7. **PhysActivity**: Aktif secara fisik dalam 30 hari terakhir atau tidak
8. **Fruits**: Mengonsumsi buah setiap hari atau tidak 
9. **Veggies**: Mengonsumsi sayur setiap hari atau tidak
10. **HvyAlcoholConsump**: Peminum berat alkohol atau bukan 
11. **AnyHealthcare**: Memiliki perlindungan kesehatan atau tidak, contohnya memiliki asuransi kesehatan
12. **GenHtlth**: Evaluasi mandiri terhadap kesehatan, skala 1-5 (1: Sangat baik, 2: Cukup Baik, 3: Baik, 4: Biasa saja, 5: Buruk)
13. **MentHlth**: Jumlah hari keadaan mental buruk dalam 30 hari terakhir (skala 0-30 hari)  
14. **PhysHlth**: Jumlah hari keadaan fisik buruk dalam 30 hari terakhir (skala 0-30 hari)
15. **DiffWalk**: Memiliki kesulitan berjalan atau menaiki tangga
16. **Sex**: (M) Male atau (F) Female
17. **Age**: 13 kategori umur (1: 18-24 tahun, 9: 60-64 tahun, 13: 80 tahun ke atas)
18. **Education**: Level edukasi skala 1-6 (1: Tidak pernah sekolah atau hanya TK, 2: SD, dst)
19. **Income**: Skala pendapatan 1-8
20. **Diabetes**: Apakah mengalami diabetes atau tidak (Kolom target)

In [1]:
# Import library di sini
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [2]:
data = pd.read_csv("diabetes.csv")

# Encode Sex column's record with Label encoding
data["Sex"].replace({"M":1,"F":0}, inplace=True)
data["Diabetes"].replace({True:1,False:0}, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50736 entries, 0 to 50735
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   HighBP                50736 non-null  float64
 1   HighChol              50736 non-null  float64
 2   BMI                   50736 non-null  float64
 3   Smoker                50736 non-null  float64
 4   Stroke                50736 non-null  float64
 5   HeartDiseaseorAttack  50736 non-null  float64
 6   PhysActivity          50736 non-null  float64
 7   Fruits                50736 non-null  float64
 8   Veggies               50736 non-null  float64
 9   HvyAlcoholConsump     50736 non-null  float64
 10  AnyHealthcare         50736 non-null  float64
 11  GenHlth               50736 non-null  float64
 12  MentHlth              50736 non-null  float64
 13  PhysHlth              50736 non-null  float64
 14  DiffWalk              50736 non-null  float64
 15  Sex                

In [3]:
data.sample(100)

,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes
31261,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0,7.0,6.0,8.0,0
29945,0.0,0.0,27.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,1,5.0,5.0,7.0,0
36013,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,2.0,5.0,6.0,0
303,1.0,0.0,37.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0,10.0,5.0,6.0,0
49282,0.0,1.0,22.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,1,8.0,6.0,6.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,1.0,0.0,29.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,1,11.0,4.0,6.0,0
42843,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1,3.0,5.0,6.0,0
34259,0.0,0.0,51.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,3.0,3.0,2.0,0.0,1,2.0,3.0,4.0,1
14692,1.0,0.0,25.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,20.0,20.0,1.0,1,9.0,6.0,3.0,1


In [4]:
X = data.drop(columns="Diabetes")
y = data["Diabetes"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=123)

# Gunakan data validasi untuk bereksperimen dengan model
# Gunakan data test untuk mengevaluasi model hanya di akhir eksperimen
df_train = pd.concat([X_train, y_train], axis=1)
df_val = pd.concat([X_val, y_val], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

_Deliverable_ yang akan dihasilkan adalah sebagai berikut:
1. berkas _notebook_ dengan format nama `PraktikumIF3270_M1_NIM1_NIM2.ipynb` untuk Bagian 1;
2. berkas _notebook_ dengan format nama `PraktikumIF3270_M2_NIM1_NIM2.ipynb` untuk Bagian 1 + Bagian 2; serta
3. berkas laporan dengan format nama `PraktikumIF3270_NIM1_NIM2.pdf` yang mencakup hal berikut:
    - hasil analisis data,
    - penanganan dari hasil analisis data,
    - justifikasi teknik-teknik yang dipilih,
    - perubahan yang dilakukan pada jawaban poin 1—5 jika ada,
    - desain eksperimen,
    - hasil eksperimen,
    - analisis dari hasil eksperimen,
    - kesimpulan,
    - pembagian tugas/kerja per anggota kelompok

Batas waktu pengumpulan:
- _Deliverable_ poin 1: Senin, 25 April 2023, pukul 12.00 WIB
- _Deliverable_ poin 2: Senin, 25 April 2023, pukul 21.00 WIB
- _Deliverable_ poin 3: Senin, 25 April 2023, pukul 21.00 WIB

## Bagian 1

#### 1. Buatlah _baseline_ dengan menggunakan model _logistic regression_.

In [5]:
baseline_model = LogisticRegression(max_iter=1000, random_state=42)
baseline_model.fit(X_train, y_train)

y_pred = baseline_model.predict(X_val)
print(f"F1 Score: {f1_score(y_pred, y_val)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_pred, y_val)}")

F1 Score: 0.23902087832973362
Confusion Matrix:
[[6895  925]
 [ 132  166]]


#### 2. Lakukan analisis data terkait hal berikut:
####    - _duplicate value_,


In [6]:

duplicateValue = data[data.duplicated()]
print("Number of duplicate value:\n ", len(duplicateValue))

Number of duplicate value:
  2329


####    - _missing value_,

In [7]:
# missing Value
missingValues = data.isnull().sum()
print("Missing values in each column:")
print(missingValues)

Missing values in each column:
HighBP                  0
HighChol                0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
Diabetes                0
dtype: int64


####    - _outlier_,

In [ ]:
# outliers
for col in data.columns:
    if data[col].dtype == "float64" or data[col].dtype == "int64":
        q1 = data[col].quantile(0.25)
        q3 = data[col].quantile(0.75)
        iqr = q3 - q1
        lowerBound = q1 - (1.5 * iqr)
        upperBound = q3 + (1.5 * iqr)

        insideRange = len(data[(data[col] >= lowerBound) & (data[col] <= upperBound)])
        outsideRange = len(data[(data[col] < lowerBound) | (data[col] > upperBound)])

        print(f"{col} have {outsideRange} outlier data")

# outliers visualization
for col in data.columns:
    if data[col].dtype == "int64" or data[col].dtype == "float64":
        plt.figure(figsize=(10, 5))
        sns.boxplot(x=data[col], color='skyblue', linewidth=2.5, fliersize=5, whis=1.5)
        plt.title(col, fontsize=16)
        plt.xlabel('values', fontsize=14)
        plt.ylabel(col, fontsize=14)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.show()

####   - _balance of data_.

In [9]:
# balance of data
uniqueValues, uniqueCount = np.unique(y, return_counts=True)
print(pd.DataFrame({"Values": uniqueValues, "Count": uniqueCount}))

   Values  Count
0       0  43790
1       1   6946


#### 3. Jelaskan rencana penanganan yang ada pada poin 2.

1. Duplicate Value
duplicate value akan ditangani dengan mempertahankan salah satu data dan menghapus yang lainnya dengan menggunakan fungsi `drop_duplicates()`

2. Missing Value
dikarenakan tidak ada missing value sama sekali pada dataset sehingga tidak perlu ada penanganan yang dilakukan.

3. Outlier
karena dataset berukuran cukup besar dan jumlah outlier cukup kecil dibandingkan dengan jumlah keseluruhan data maka untuk menangani outlier dilakukan dengan cara trimming atau menghilangkan data yang mengandung outlier

4. Balance of Data
karena terdapat 43790 nilai "False" dan 6946 nilai "True", maka akan dilakukan teknik Oversampling dan Undersampling. Pemilihan teknik yang digunakan akan didasarkan pada matriks evaluasi F1-Score, teknik dengan nilai F1-Score yang lebih tinggi akan digunakan sebagai teknik untuk menangani ketidak seimbangan data.


#### 4. Jelaskan teknik _encoding_ yang digunakan terhadap data yang disediakan apabila dilakukan, disertai dengan alasan.

Kami telah mencetak contoh _record_ yang tersimpan dalam data menggunakan perintah _data.head(5)_ dan juga informasi terkait tiap atribut menggunakan perintah _data.info()_. Hasilnya menunjukkan bahwa hanya akan dilakukan dua kali proses encoding pada kolom **Sex** (nilainya berupa objek "M" dan "F") dan kolom target **Diabetes** (nilainya berupa bool _true_ dan _false_). Oleh karena itu, kami menggunakan teknik encoding yang disebut sebagai _label encoding_. Metode tersebut akan memetakan tiap nilai kategorik menjadi nilai numerik yang unik mulai dari 0, 1, 2, 3, dst.

Terdapat kelebihan dan kekurangan di dalam penggunaan teknik _label encoding_. Kelebihannya adalah teknik tersebut sangat mudah untuk digunakan. Akan tetapi, pada kasus tertentu, misalkan apabila sebuah kolom memiliki nilai lebih dari 3 kategori, maka akan terbentuk semacam nilai berurutan. Hal itu dapat menjadi sisi kekurangan dari penggunaan _label encoding_, karena dapat memungkinkan sebuah model untuk memberikan bobot yang tidak diinginkan pada nilai numerik yang berbeda. Namun, karena kolom **Sex** dan **Diabetes** hanya memiliki 2 nilai kategori, maka kekurangan dari _label encoding_ tidak akan berdampak pada model yang dibuat, karena nilai kardinalitasnya rendah.

#### 5. Buatlah desain eksperimen dengan menentukan hal berikut:
####    - tujuan eksperimen
####    - variabel dependen dan independen
####    - strategi eksperimen
####    - skema validasi

**Tujuan Eksperimen** : Membuat sebuah model yang dapat mengklasifikasikan apakah seseorang dengan kondisi kesehatan tertentu mulai dari nilai tekanan darah, kolesterol, _Body Mass Index_ (BMI), perokok, pernah menderita struk, riwayat penyakit jantung koroner dan serangan jantung, aktivitas fisik, konsumsi buah, mengkonsumsi sayur, peminum alkohol, pemilik perlindungan kesehatan, evaluasi mandiri kesehatan, jumlah hari dengan mental dan keadaan fisik yang buruk, kesulitan berjalan, jenis kelamin,  umur, taraf pendidikan, dan pemasukan beresiko untuk terkena penyakit diabetes atau tidak. 

**Variabel Dependen** :
 - Diabetes (biner: 0 untuk Tidak Diabetes, 1 untuk Diabetes)

**Variabel Independen** : 
- HighBP: Tekanan darah tinggi (biner: 0 untuk Tidak, 1 untuk Ya)
- HighChol: Kolesterol tinggi (biner: 0 untuk Tidak, 1 untuk Ya)
- BMI: Indeks Massa Tubuh (kontinu)
- Smoker: Perokok (biner: 0 untuk Tidak, 1 untuk Ya)
- Stroke: Riwayat stroke (biner: 0 untuk Tidak, 1 untuk Ya)
- HeartDiseaseorAttack: Penyakit jantung atau serangan jantung (biner: 0 untuk Tidak, 1 untuk Ya)
- PhysActivity: Aktivitas fisik (biner: 0 untuk Tidak, 1 untuk Ya)
- Fruits: Konsumsi buah-buahan (kontinu)
- Veggies: Konsumsi sayuran (kontinu)
- HvyAlcoholConsump: Konsumsi alkohol berat (biner: 0 untuk Tidak, 1 untuk Ya)
- AnyHealthcare: Pelayanan kesehatan yang tersedia (biner: 0 untuk Tidak, 1 untuk Ya)
- GenHlth: Kesehatan umum (kontinu)
- MentHlth: Kesehatan mental (kontinu)
- PhysHlth: Kesehatan fisik (kontinu)
- DiffWalk: Kesulitan berjalan (kontinu)
- Sex: Jenis kelamin (biner: 0 untuk Female, 1 untuk Male)
- Age: Usia (kontinu)
- Education: Tingkat pendidikan (kontinu)
- Income: Pendapatan (kontinu)

**Strategi Eksperimen**:
1. Pertama: Kami akan melakukan _data preprocessing_ (sesuai dengan apa yang termuat pada poin di atasnya). 
2. Kedua: Kami juga melakukan proses lain seperti penggunaan data scaling menggunakan _min-max scaler_ atau semacamnya, serta telah dilakukan pula proses _label encoding_ untuk mengubah nilai dari suatu kolom yang formatnya tidak sesuai menjadi berformat int64 dengan nilai yang unik.
3. Ketiga : Kami akan membangun beberapa model yang tentunya dapat digunakan sebagai kakas _binary classification_ seperti: 
    - Logistic regression
    - Support vector machines
    - Naive bayes
    - Neareset neighbor
    - Decision trees
    - Neural networks
4. Keempat : Kami akan memvalidasi model yang telah dibuat dengan skema validasi pada poin di bawah ini.

**Strategi Validasi**:
Kami akan menggunakan skema validasi K-Fold untuk membagi _data training_ menjadi beberapa bagian. Kemudian akan digunakan matriks evaluasi F1-Score, serta model dengan nilai F1-Score tertinggi akan dipilih sebagai model terbaik.

## Bagian 2

In [10]:
df = data.copy()
df.head(5)

,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes
0,0.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0,9.0,6.0,8.0,0
1,0.0,0.0,24.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,1,13.0,6.0,8.0,0
2,1.0,0.0,29.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,20.0,5.0,0.0,0,12.0,6.0,8.0,0
3,0.0,0.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,30.0,0.0,0.0,1,1.0,5.0,7.0,0
4,0.0,0.0,33.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,5.0,30.0,15.0,1.0,0,11.0,4.0,3.0,0


### Data Preprocessing

#### Data Scaling

In [33]:
# Proses scaling menggunakan MinMaxScaler
scaler = MinMaxScaler()
for col in df.columns:
    if df[col].dtype == "int64" or df[col].dtype == "float64":
        scaled_data = scaler.fit_transform(df[col].values.reshape(-1, 1))
        df[col] = scaled_data.flatten()

df.head(5)

,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes
0,0.0,1.0,0.354839,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.00,0.000000,0.000000,0.0,0.0,0.666667,1.0,1.000000,0.0
1,0.0,0.0,0.354839,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.25,0.000000,0.000000,1.0,1.0,1.000000,1.0,1.000000,0.0
2,1.0,0.0,0.516129,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.25,0.666667,0.166667,0.0,0.0,0.916667,1.0,1.000000,0.0
3,0.0,0.0,0.483871,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.25,1.000000,0.000000,0.0,1.0,0.000000,0.8,0.857143,0.0
4,0.0,0.0,0.645161,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.00,1.000000,0.500000,1.0,0.0,0.833333,0.6,0.285714,0.0


#### Drop duplicate values

In [12]:
# Handling duplicate values
print(f"{50*'='}")
print(f"Before drop duplicate values: {df.shape}")

df = df.drop_duplicates()

print(f"After drop duplicate values: {df.shape}")
print(f"{50*'='}")

Before drop duplicate values: (50736, 20)
After drop duplicate values: (48407, 20)


#### Drop Outlier Values

In [13]:
# Handling outlier values (hanya dilakukan pada kolom BMI, didasarkan pada domain knowledge)
print(f"{50*'='}")
print(f"Before trim outlier values: {df.shape}")

q1 = df["BMI"].quantile(0.25)
q3 = df["BMI"].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)

df = df[(df["BMI"] >= lower_bound) & (df["BMI"] <= upper_bound)]

print(f"After trim outlier values: {df.shape}")
print(f"{50*'='}")

Before trim outlier values: (48407, 20)
After trim outlier values: (47254, 20)


In [14]:
# Encode kolom dengan label encoding (telah dilakukan sebelumnya)

df["Sex"].replace({"M":1,"F":0}, inplace=True)
df["Diabetes"].replace({True:1,False:0}, inplace=True)
df.sample(5)

,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes
8658,1.0,0.0,0.200000,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.25,0.000000,0.0,0.0,1.0,0.416667,0.8,0.714286,0.0
42382,0.0,0.0,0.294118,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.75,1.000000,0.0,0.0,1.0,0.500000,0.4,0.428571,0.0
11612,1.0,0.0,0.152941,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.00,0.333333,0.0,0.0,1.0,0.000000,0.8,1.000000,0.0
17935,0.0,0.0,0.117647,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.25,0.000000,0.0,0.0,0.0,0.250000,0.8,0.571429,0.0
2651,0.0,0.0,0.211765,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.00,0.000000,0.0,0.0,1.0,0.333333,1.0,1.000000,0.0


In [15]:
# Bagi menjadi data train, validation dan test
X = df.drop(columns="Diabetes")
y = df["Diabetes"].copy()

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=2024)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=2024)

print(f"{50*'='}")
print("Train set size:", X_train.shape[0])
print("Validation set size:", X_val.shape[0])
print("Test set size:", X_test.shape[0])
print(f"{50*'='}")

Train set size: 37803
Validation set size: 4725
Test set size: 4726


#### Data Sampling

In [16]:
# Undersampling
underSampler = RandomUnderSampler(sampling_strategy="majority", random_state=2024)
underSampler_X_train, underSampler_y_train = underSampler.fit_resample(X_train, y_train)

print(f"{50*'='}")

print(f"Number sample: {len(underSampler_X_train)}")

underSamplerLogistic = LogisticRegression(max_iter=1000, random_state=2024)
underSamplerLogistic.fit(underSampler_X_train, underSampler_y_train)
underSampler_y_pred = underSamplerLogistic.predict(X_test)

print(f"Accuracy: {accuracy_score(underSampler_y_pred, y_test)}")
print(f"F1 Score: {f1_score(underSampler_y_pred, y_test)}")
print(f"Confusion Matrix:\n{confusion_matrix(underSampler_y_pred, y_test)}")

print(f"{50*'='}")

Number sample: 10398
Accuracy: 0.7160389335590351
F1 Score: 0.4269854824935952
Confusion Matrix:
[[2884  154]
 [1188  500]]


In [17]:
# Oversampling
overSampler = RandomOverSampler(sampling_strategy="minority", random_state=2024)
overSampler_X_train, overSampler_y_train = overSampler.fit_resample(X_train, y_train)

print(f"{50*'='}")

print(f"Number sample: {len(overSampler_X_train)}")

overSamplerLogistic = LogisticRegression(max_iter=1000, random_state=2024)
overSamplerLogistic.fit(overSampler_X_train, overSampler_y_train)
overSampler_y_pred = overSamplerLogistic.predict(X_test)

print(f"Accuracy: {accuracy_score(overSampler_y_pred, y_test)}")
print(f"F1 Score: {f1_score(overSampler_y_pred, y_test)}")
print(f"Confusion Matrix:\n{confusion_matrix(overSampler_y_pred, y_test)}")

print(f"{50*'='}")

Number sample: 65208
Accuracy: 0.7147693609818028
F1 Score: 0.42442356959863364
Confusion Matrix:
[[2881  157]
 [1191  497]]


In [18]:
# SMOTE
smoteSampler = SMOTE(random_state=2024)
smoteSampler_X_train, smoteSampler_y_train = smoteSampler.fit_resample(X_train, y_train)

print(f"{50*'='}")

print(f"Number sample: {len(smoteSampler_X_train)}")

smoteSamplerLogistic = LogisticRegression(max_iter=1000, random_state=2024)
smoteSamplerLogistic.fit(smoteSampler_X_train, smoteSampler_y_train)
smoteSampler_y_pred = smoteSamplerLogistic.predict(X_test)

print(f"Accuracy: {accuracy_score(smoteSampler_y_pred, y_test)}")
print(f"F1 Score: {f1_score(smoteSampler_y_pred, y_test)}")
print(f"Confusion Matrix:\n{confusion_matrix(smoteSampler_y_pred, y_test)}")

print(f"{50*'='}")

Number sample: 65208
Accuracy: 0.7173085061362675
F1 Score: 0.42114384748700173
Confusion Matrix:
[[2904  168]
 [1168  486]]


### Model

#### Grid Search : Penentuan hyperparameter terbaik



In [19]:
# Decision Tree Learning

param_grid = {
    'max_depth': [5, 10, 15],
    'criterion': ['gini', 'entropy'],
}

dtl = DecisionTreeClassifier(random_state=2024, splitter='best')
gridSearch_dtl = GridSearchCV(estimator=dtl, param_grid=param_grid, n_jobs=-1, verbose=2)
gridSearch_dtl.fit(smoteSampler_X_train, smoteSampler_y_train)
gridSearch_y_pred = gridSearch_dtl.predict(X_test)

print(f"Parameter: {gridSearch_dtl.best_params_}") 
print(f"Accuracy: {accuracy_score(gridSearch_y_pred, y_test)}")
print(f"F1 Score: {f1_score(gridSearch_y_pred, y_test)}")
print(f"Confusion Matrix:\n{confusion_matrix(gridSearch_y_pred, y_test)}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Parameter: {'criterion': 'entropy', 'max_depth': 15}
Accuracy: 0.8015234870926788
F1 Score: 0.34951456310679613
Confusion Matrix:
[[3536  402]
 [ 536  252]]


In [20]:
# Naive Bayes

param_grid = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

nb = GaussianNB()
gridSearch_nb = GridSearchCV(estimator=nb, param_grid=param_grid, n_jobs=-1, verbose=2)
gridSearch_nb.fit(smoteSampler_X_train, smoteSampler_y_train)
gridSearch_y_pred_nb = gridSearch_nb.predict(X_test)

print("Parameter:", gridSearch_nb.best_params_)
print("Accuracy:", accuracy_score(gridSearch_y_pred_nb, y_test))
print("F1 Score:", f1_score(gridSearch_y_pred_nb, y_test))
print("Confusion Matrix:\n", confusion_matrix(gridSearch_y_pred_nb, y_test))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Parameter: {'var_smoothing': 1.2328467394420658e-05}
Accuracy: 0.7156157426999576
F1 Score: 0.40372670807453415
Confusion Matrix:
 [[2927  199]
 [1145  455]]


In [21]:
# Logistic Regression

param_grid = {
    "solver": ["liblinear", "lbfgs", "newton-cg", "sag", "saga"]
}

lr = LogisticRegression(max_iter=1000, random_state=2024)
gridSearch_lr = GridSearchCV(estimator=lr, param_grid=param_grid, n_jobs=-1, verbose=2)
gridSearch_lr.fit(smoteSampler_X_train, smoteSampler_y_train)
gridSearch_y_pred = gridSearch_lr.predict(X_test)

print(f"Parameter: {gridSearch_lr.best_params_}")
print(f"Accuracy: {accuracy_score(gridSearch_y_pred, y_test)}")
print(f"F1 Score: {f1_score(gridSearch_y_pred, y_test)}")
print(f"Confusion Matrix:\n{confusion_matrix(gridSearch_y_pred, y_test)}")

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Parameter: {'solver': 'lbfgs'}
Accuracy: 0.7173085061362675
F1 Score: 0.42114384748700173
Confusion Matrix:
[[2904  168]
 [1168  486]]


In [28]:
# Random Forest

param_grid = {
    'n_estimators': [10, 20, 30, 40, 50],
    'max_depth': [25, 40, 70, 90, 100],
    'criterion': ['gini', 'entropy'],
}

rf = RandomForestClassifier(random_state=2024)
gridSearch_rf = GridSearchCV(estimator=rf, param_grid=param_grid, n_jobs=-1, verbose=2)
gridSearch_rf.fit(smoteSampler_X_train, smoteSampler_y_train)
gridSearch_y_pred = gridSearch_rf.predict(X_test)

print(f"Parameter: {gridSearch_rf.best_params_}")
print(f"Accuracy: {accuracy_score(gridSearch_y_pred, y_test)}")
print(f"F1 Score: {f1_score(gridSearch_y_pred, y_test)}")
print(f"Confusion Matrix:\n{confusion_matrix(gridSearch_y_pred, y_test)}")

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Parameter: {'criterion': 'gini', 'max_depth': 40, 'n_estimators': 50}
Accuracy: 0.841726618705036
F1 Score: 0.3175182481751825
Confusion Matrix:
[[3804  480]
 [ 268  174]]


In [23]:
# Support Vector Machine

param_grid = {
    'kernel': ['rbf', 'linear', 'poly']
}

svm = SVC(random_state = 2024)
gridSearch_svm = GridSearchCV(estimator=svm, param_grid=param_grid, n_jobs=-1, verbose=2)
gridSearch_svm.fit(smoteSampler_X_train, smoteSampler_y_train)
gridSearch_y_pred_svm = gridSearch_svm.predict(X_test)

print("Parameter:", gridSearch_svm.best_params_)
print("Accuracy:", accuracy_score(gridSearch_y_pred_svm, y_test))
print("F1 Score:", f1_score(gridSearch_y_pred_svm, y_test))
print("Confusion Matrix:\n", confusion_matrix(gridSearch_y_pred_svm, y_test))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Parameter: {'kernel': 'rbf'}
Accuracy: 0.7179432924248836
F1 Score: 0.41147902869757175
Confusion Matrix:
 [[2927  188]
 [1145  466]]


In [24]:
# K Nearest Neighbor

k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)

knn = KNeighborsClassifier()
gridSearch_knn = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1)
gridSearch_knn.fit(smoteSampler_X_train, smoteSampler_y_train)
gridSearch_y_pred_knn = gridSearch_knn.predict(X_test)

print("Parameter:", gridSearch_knn.best_params_)
print("Accuracy:", accuracy_score(gridSearch_y_pred_knn, y_test))
print("F1 Score:", f1_score(gridSearch_y_pred_knn, y_test))
print("Confusion Matrix:\n", confusion_matrix(gridSearch_y_pred_knn, y_test))

Fitting 10 folds for each of 30 candidates, totalling 300 fits


Parameter: {'n_neighbors': 1}
Accuracy: 0.770207363520948
F1 Score: 0.3047375160051216
Confusion Matrix:
 [[3402  416]
 [ 670  238]]


In [29]:
# Neural Networks

def buildModel(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 128, activation = 'relu'))
    classifier.add(Dense(units = 64, activation = 'relu'))
    classifier.add(Dense(units = 32, activation = 'relu'))
    classifier.add(Dense(units = 1, activation = 'linear'))
    classifier.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_absolute_error'])
    return classifier

param_grid = {
    'batch_size': [10, 20, 40, 60, 80, 100],
    'epochs': [10, 50, 100],
    'optimizer': ['adam', 'rmsprop']
}

nn = KerasRegressor(build_fn=buildModel, verbose=0)
gridSearch_nn = GridSearchCV(estimator=nn, param_grid=param_grid, n_jobs=-1, verbose=2)
gridSearch_nn.fit(smoteSampler_X_train, smoteSampler_y_train)
gridSearch_y_pred_nn = gridSearch_nn.predict(X_test)

print("Parameter:", gridSearch_nn.best_params_)
print("Accuracy:", accuracy_score(gridSearch_y_pred_nn, y_test))
print("F1 Score:", f1_score(gridSearch_y_pred_nn, y_test))
print("Confusion Matrix:\n", confusion_matrix(gridSearch_y_pred_nn, y_test))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


ValueError: 
All the 180 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\ASUS\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\ASUS\AppData\Local\Programs\Python\Python39\lib\site-packages\scikeras\wrappers.py", line 770, in fit
    self._fit(
  File "c:\Users\ASUS\AppData\Local\Programs\Python\Python39\lib\site-packages\scikeras\wrappers.py", line 925, in _fit
    X, y = self._initialize(X, y)
  File "c:\Users\ASUS\AppData\Local\Programs\Python\Python39\lib\site-packages\scikeras\wrappers.py", line 862, in _initialize
    self.model_ = self._build_ke

#### Model Training

In [30]:
rf_1 = RandomForestClassifier(criterion="gini", max_depth=25, n_estimators=40, random_state=2024)
rf_2 = RandomForestClassifier(criterion="gini", max_depth=25, n_estimators=40, random_state=2025)
rf_3 = RandomForestClassifier(criterion="gini", max_depth=25, n_estimators=40, random_state=2026)
final_estimator = LogisticRegression(solver="liblinear", max_iter=1000, random_state=2024)

stacking_ensemble = StackingClassifier(
    estimators=[
        ("rf_1",rf_1),
        ("rf_2", rf_2),
        ("rf_3", rf_3),
    ],
    final_estimator=final_estimator
)
stacking_ensemble.fit(smoteSampler_X_train, smoteSampler_y_train)
stacking_ensemble_pred = stacking_ensemble.predict(X_val)

print(f"Accuracy: {accuracy_score(stacking_ensemble_pred, y_val)}")
print(f"F1 Score: {f1_score(stacking_ensemble_pred, y_val)}")
print(f"Confusion Matrix:\n{confusion_matrix(stacking_ensemble_pred, y_val)}")

Accuracy: 0.8391534391534392
F1 Score: 0.3770491803278688
Confusion Matrix:
[[3735  449]
 [ 311  230]]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c384c4b3-0606-4ac0-935c-3feaec40b0ba' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

#### Model Testing

In [32]:
print(f"{50*'='}")
stacking_ensemble_test = stacking_ensemble.predict(X_test)

print(f"Accuracy: {accuracy_score(stacking_ensemble_test, y_test)}")
print(f"F1 Score: {f1_score(stacking_ensemble_test, y_test)}")
print(f"Confusion Matrix:\n{confusion_matrix(stacking_ensemble_test, y_test)}")
print(f"{50*'='}")

Accuracy: 0.8322048243757935
F1 Score: 0.3364016736401674
Confusion Matrix:
[[3732  453]
 [ 340  201]]
